In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import json
import pickle

In [2]:
soup = BeautifulSoup(open('0005.html'), "html.parser")

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<title>0005 - verybi_verytired - The Umbrella Academy TV</title>
<style type="text/css">p.message{text-align:center}.meta h1{font-size:1.5em;text-align:center}.meta h2{font-size:1.25em;text-align:center}.meta h2{page-break-before:always}.meta .byline{text-align:center}.meta dl.tags{border:1px solid;padding:1em}.meta dd{margin:-1em 0 0 10em}.meta .endnote-link{font-size:.8em}#chapters{font-family:"Nimbus Roman No9 L","Times New Roman",serif;padding:1em}.userstuff{font-family:"Nimbus Roman No9 L","Times New Roman",serif;padding:1em}.toc-heading{display:none}</style>
</head>
<body>
<div id="preface">
<h2 class="toc-heading">Preface</h2>
<p class="message">
<b>00.05</b><br/>
    Posted originally on the <a href="http://archiveofo

json data = 
Title
AO3 link
Rating
Warning
Audience Category
Fandom(s)
Characters
Tags
Series
Collections
Publish Date
Text Length

In [151]:
def extractStoryURL(soup_obj): 
    #code to extract story URL
    i = 0
    for element in soup_obj.find(class_="message"):
        i += 1
        if i == 7:
            url = element['href']

        else:
            continue
            
    return url

In [152]:
webAddress = extractStoryURL(soup)

In [153]:
webAddress

'http://archiveofourown.org/works/18632218'

In [176]:
def extractAuthor(soup_obj):
    #code to extract author and fandom info
    for info in soup_obj.title:
        incomplete_title, author, incomplete_fandom = str(info).split('-')
        author = author.strip()
    return author
    

def extractTitleStory(soup_obj):
    #code to extract title and story text
    i = 0
    story_text = ""
    for element in soup_obj.find(id="chapters"):
        if i == 1:
            title = element.string
        elif i == 3:
            story_ptag_text = element.find_all('p')
            for obj in story_ptag_text:
                if obj.string == None:
                    continue
                else:
                    story_text += str(obj.string) + " "
        i += 1
    return story_text
        
def extractMetadata(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    fandom_text = ""
    additionalTags_text = ""
    for element in soup_obj.find(class_="tags"):
        i += 1
        if i == 4:
            rating = element.string
        elif i == 8:
            archive_warning = element.string
        elif i == 12:
            category = element.string
        elif i == 16:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += obj.string + ", "
            fandom_text = fandom_text[:-2]
        elif i == 20:
            character = element.string
        elif i == 24:
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += obj.string + ", "
            additionalTags_text = additionalTags_text[:-2]
        elif i == 28:
            series_name = element.find('a').string
        elif i == 32:
            collections = element.string
        elif i == 36:
            pub_stat = str(element.string).strip()
            publish_date = pub_stat[11:21]
            word_count = pub_stat[37:40]
        else:
            continue
    return fandom_text, additionalTags_text, rating, archive_warning, category, series_name, collections, publish_date, word_count

def extractStoryURL(soup_obj): 
    #code to extract story URL
    i = 0
    for element in soup_obj.find(class_="message"):
        i += 1
        if i == 7:
            url = element['href']

        else:
            continue
    return url

def createAO3dict(filelist):
    i = 0
    ao3_dict = {}
    for element in filelist:
        soup = BeautifulSoup(open(element), "html.parser")
        url = extractStoryURL(soup)
        fandom_text, additionalTags_text, rating, archive_warning, category, series_name, collections, pub_date, word_count = extractMetadata(soup)
        author = extractAuthor(soup)
        title = extractTitleStory(soup)
        
        story_dict = {'title': title,
            'author': author,
            'story_text': story_text, 
            'rating': rating, 
            'archive_warning': archive_warning, 
            'category': category, 
            'fandom_text': fandom_text, 
            'character': character, 
            'additionalTags_text': additionalTags_text, 
            'series_name': series_name, 
            'collections': collections, 
            'pub_date': pub_date, 
            'word_count': word_count,
            'url': url
           }
        ao3_dict['i'] = story_dict
        i += 1
    return ao3_dict

In [144]:
fanfics = ['Fanfic_HTML/13 Heirs.html', 'Fanfic_HTML/In Which A Wizard Comes.html', 'Fanfic_HTML/Sweet Basil and Italian.html', 
           'Fanfic_HTML/5 6 7 The Umbrella.html', 'Fanfic_HTML/In Which A Wizard Is.html', 'Fanfic_HTML/Swine and Forests.html',
           'Fanfic_HTML/50 Truths of the.html', 'Fanfic_HTML/In Which Calcifer Meets.html', 'Fanfic_HTML/Symbol of Reform.html',
           'Fanfic_HTML/A Bakers Proposal.html', 'Fanfic_HTML/In Which Howl Attracts.html', 'Fanfic_HTML/Sympathy.html']

In [177]:
ao3_dictionary = createAO3dict(fanfics)

UnboundLocalError: local variable 'publish_date' referenced before assignment

In [ ]:
ao3_dictionary

In [140]:
ao3_df = pd.DataFrame.from_dict(ao3_data)
ao3_df

0
title                                                            00.05
author                                                verybi_verytired
story_text           He was hungry. Always so hungry. He wasn’t exa...
rating                                               General Audiences
archive_warning                     Choose Not To Use Archive Warnings
category                                                           Gen
fandom_text          The Umbrella Academy (TV), The Umbrella Academ...
character                     Number Five | The Boy (Umbrella Academy)
additionalTags_text  Eating Disorders, Eating Disorder Not Otherwis...
series_name                                   Angsty Headcanon Ficlets
collections                                    Bad Things Happen Bingo
pub_date                                                    2019-04-28
word_count                                                         327
url                          http://archiveofourown.org/works/18632218

In [137]:
#code from https://www.geeksforgeeks.org/how-to-convert-python-dictionary-to-json/    
# Serializing json    
json_object = json.dumps(ao3_data, indent = 4)   
print(json_object)

{
    "title": "00.05",
    "author": "verybi_verytired",
    "story_text": "He was hungry. Always so hungry. He wasn\u2019t exactly sure why he was doing this to himself. He didn\u2019t want or need to lose weight, there was always plenty of food, and everyone else was well fed and happy. So why couldn\u2019t he bring himself to eat? During the apocalypse, he barely ate. It was rare that he was full, and when he was full he tended to get sick, his stomach not used to the amount of food in it. Within a year or two of arriving in the ruins of the city, he was used to passing out from a lack of food or water. After he started working as a time travelling assassin he thought it would get better. He was wrong. For some reason, even when literally surrounded by food, he found himself sick at the idea of eating it. Food tasted too good, too strong and it felt weird in his mouth, the textures too much for him to handle. The weight of it in his stomach made him want to curl up or get sick \u20

In [138]:
ao3_df = pd.read_json(json_object, orient='columns')

ValueError: If using all scalar values, you must pass an index

In [114]:
ao3_df

0
title                                                            00.05
author                                                verybi_verytired
story_text           He was hungry. Always so hungry. He wasn’t exa...
rating                                               General Audiences
archive_warning                     Choose Not To Use Archive Warnings
category                                                           Gen
fandom_text          [The Umbrella Academy (TV), The Umbrella Acade...
character                     Number Five | The Boy (Umbrella Academy)
additionalTags_text  [Eating Disorders, Eating Disorder Not Otherwi...
series_name                                   Angsty Headcanon Ficlets
collections                                    Bad Things Happen Bingo
pub_date                                                    2019-04-28
word_count                                                         327
url                          http://archiveofourown.org/works/18632218

In [116]:
ao3_df.pivot(index=[0])

KeyError: 'None of [None] are in the columns'

In [63]:
print(fandom_text)

['The Umbrella Academy (TV)', 'The Umbrella Academy (Comics)']
